In [1]:
import tensorflow as tf
tf.__version__

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.0'

In [2]:
from tensorflow.contrib import keras

In [3]:
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
import time
import os

np.random.seed(1337)

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from numpy import nan
import sys
import datetime

now = datetime.datetime.now

In [5]:
from matplotlib import pyplot as plt

%matplotlib inline

In [6]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [7]:
img_rows, img_cols = IMG_WIDTH, IMG_HEIGHT

if K.image_data_format() == 'channels_first':
    shape_ord = (3, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)

In [8]:
from keras.applications import vgg16
from keras.layers import Input

# build the VGG16 network with ImageNet weights
vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input((224, 224, 3)),classes=5)
print('Model loaded.')

Model loaded.


In [9]:
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization

In [10]:
for l in vgg16.layers:
    l.trainable = False

In [11]:
x = Flatten(input_shape=vgg16.output.shape)(vgg16.output)
x = Dense(2048, activation='relu', name='ft_fc1')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='ft_fc2')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(5, activation = 'softmax')(x)

In [12]:
from keras.models import Model

model = Model(inputs=vgg16.input, outputs=predictions)

#compile the model
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
train_datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

In [14]:
train_generator = train_datagen.flow_from_directory(
        '../data/train2',
        target_size=(224, 224),
        batch_size=32, #changed from 1, change back if needed
        class_mode='categorical')

Found 1506 images belonging to 5 classes.


In [33]:
validation_datagen = ImageDataGenerator()

In [34]:
validation_generator = validation_datagen.flow_from_directory(
        '../data/validation2',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 502 images belonging to 5 classes.


In [35]:
test_datagen = ImageDataGenerator()

In [36]:
test_generator = test_datagen.flow_from_directory(
        '../data/test2',
        target_size=(224,224),
        batch_size=32,
        class_mode='categorical')

Found 224 images belonging to 5 classes.


In [37]:
from keras.preprocessing import image
from keras.callbacks import TensorBoard

In [38]:
model.fit_generator(
        train_generator,
        steps_per_epoch=47,
        epochs=40,
        validation_data=validation_generator,
        validation_steps=16,
        use_multiprocessing=True)
#         callbacks=[TensorBoard(log_dir='./tf_logs', histogram_freq=0, 
#                                write_graph=True, write_images=True, 
#                                embeddings_freq=10, 
#                                embeddings_layer_names=['block1_conv2', 
#                                                        'block5_conv1', 
#                                                        'ft_fc1'], 
#                                embeddings_metadata=None)])

Epoch 1/40
48/47 [==============================] - 61s 1s/step - loss: 0.3562 - acc: 0.8681 - val_loss: 1.0876 - val_acc: 0.6693
Epoch 2/40
48/47 [==============================] - 52s 1s/step - loss: 0.3803 - acc: 0.8580 - val_loss: 0.9935 - val_acc: 0.6793
Epoch 3/40
48/47 [==============================] - 53s 1s/step - loss: 0.4386 - acc: 0.8188 - val_loss: 0.9975 - val_acc: 0.6693
Epoch 4/40
48/47 [==============================] - 52s 1s/step - loss: 0.3332 - acc: 0.8784 - val_loss: 0.9632 - val_acc: 0.6912
Epoch 5/40
48/47 [==============================] - 51s 1s/step - loss: 0.4216 - acc: 0.8638 - val_loss: 0.9805 - val_acc: 0.6952
Epoch 6/40
48/47 [==============================] - 52s 1s/step - loss: 0.3253 - acc: 0.8810 - val_loss: 1.0033 - val_acc: 0.6614
Epoch 7/40
48/47 [==============================] - 51s 1s/step - loss: 0.3797 - acc: 0.8660 - val_loss: 1.0472 - val_acc: 0.6833
Epoch 8/40
48/47 [==============================] - 51s 1s/step - loss: 0.3650 - acc: 0.87

In [21]:
model_history = _

In [39]:
model.evaluate_generator(validation_generator,steps=15)

[1.3126184980707813, 0.67729083760326125]

In [23]:
import cv2

In [24]:
import pandas as pd
df = pd.read_csv('../data/test2/xy.txt')
df.head()

,file_name,order
0,picture194.jpg,Ephemeridae
1,picture1600.jpg,Heptageniidae
2,photo_1344371.jpg,Ephemeridae
3,picture383.jpg,Ephemerellidae
4,picture814.jpg,Heptageniidae


In [25]:
t = test_generator.class_indices

In [26]:
y_cat = np_utils.to_categorical([t[i] for i in df.order], 5)
y_cat

array([[ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

In [27]:
iml = []
for f,o in zip(df.file_name,df.order):
    i_path = '../data/test2/{}/{}'.format(o,f)
    iml.append(cv2.resize(cv2.imread(i_path,1),(224,224),interpolation = cv2.INTER_AREA))

In [28]:
y_cat = np.array(y_cat)
y_true = []
y_pred = []


iml = np.stack(iml)

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(iml,y_cat,
        batch_size=32)

In [29]:
pred = model.predict(iml)

In [30]:
inv_map = {v: k for k, v in t.items()}
y_pred = [inv_map[np.argmax(i)] for i in pred]

In [31]:
y_true = [inv_map[np.argmax(i)] for i in y_cat]

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred))

                precision    recall  f1-score   support

      Baetidae       0.41      0.29      0.34        31
Ephemerellidae       0.72      0.74      0.73        80
   Ephemeridae       0.81      0.75      0.78        28
 Heptageniidae       0.60      0.59      0.60        64
 Leptohyphidae       0.29      0.43      0.35        21

   avg / total       0.61      0.61      0.61       224

